# Portfolio Learning Item

### What is a portfolio

We'll now learn about the final representation and class in our portfolio manager, the portfolio. We've discussed how individuals or institutions can have multiple accounts for different purposes. A portfolio is a collection of these accounts, but also is any collection of investments/financial assets, including stock, bonds, derivatives, ETFs, etc. Accounts are technically a portfolio, but large firms will usually have multiple accounts with different trading strategies. The collection of these accounts represents their portfolio.

For example, I could have account *Tech Strategy* which contains the positions:

***2,000*** shares of ***MSFT USD***,
***200*** shares of ***AAPL USD***, and 
***100*** shares of ***TSLA USD***

And an account *Healthcare Strategy* which contains the positions:

***1,000*** shares of ***UNH USD***,
***200*** shares of ***HCA USD***, and 
***100*** shares of ***MSFT USD***

My entire portfolio is the union of these two accounts. It's important to note that accounts within a portfolio don't need to be mutually exclusive in their positions.

### Problem Definition

We want to build a class that allows for the management of a portfolio. We should be able to construct a portfolio using a set of accounts and a portfolio name. We should be able to query the portfolio object retrieving the name of the portfolio, a subset of accounts based on a set of account names or return a collection of all accounts. We should also be able to remove/add accounts based on a set of account names or account objects respectively. Removals should ignore non-existent names & additions can replace existing accounts.

- Allow for a portfolio to be created with a set of accounts and a portfolio name
- Allow for querying of the current portfolio's name
- Allow for querying of all accounts within the portfolio
- Allow for the querying for accounts in the portfolio that match the filter criteria, returning an iterable
    - The user can give a set of account names. Any accounts that match these names should be returned
    - The user can give a set of security names or objects. Any accounts that contain positions which match the set should be returned
    - The user can combine both queries creating a dual filter. Accounts that pass both filters should be returned.
- Allow for accounts to be added to the portfolio with a set of account objects. Incoming accounts can replace existing accounts
- Allow for the removal of accounts from the portfolio with a set of account names. Names not in the portfolio should be ignored.

### Provided Tools

#### *Data Source*

For this section no data generators are provided

#### *Solution Interface*

Your solution will need to follow the interface provided in the lab. Below is a snippet of the interface for securities that you can inherit from. The methods that will be tested are displayed & will need to be overwritten with your implementation. You're free to add more methods then what is displayed in the interface!

```python
#filename interfaces.portfolio_interface.py
#Portfolio Class Interface

from typing import Set, Iterable
from .account_interface import AccountInterface
from .security_interface import SecurityInterface
class PortfolioInterface():
    def __init__(self, portfolioName: str, accounts: Set[AccountInterface]) -> None:
        pass

    def get_all_accounts(self) -> Iterable[AccountInterface]:
        pass

    def get_accounts(self, account_names_filter:Set[str], securities_filter:Set) -> Iterable[AccountInterface]:
        pass 

    def add_accounts(self, accounts: Set[AccountInterface]) -> None:
        pass

    def remove_accounts(self, account_names: Set[str]) -> None:
        pass

```

#### *Testing*

Once you have completed & saved your solution you can run the test file to validate that your solution works as expected. For the test to run the following need to be true.
- Saved code to file **implementations/portfolio_solution.py**
- Create a class with the name **Portfolio** that inherits from **PortfolioInterface**

### Stretch Goals

If you complete your class & have a solution with valid tests try completing the following stretch goals 

- Update your class init to handle the account set being optional
- Add custom implementation for class's __str__ method.
- Add a method to remove clear the entire portfolio of all accounts
- Develop tests for the new methods created

In [ ]:
#Uncomment line above & run cell to save solution
#TODO Define class that implements portFolioInterface & allows for the management of a portfolio

In [ ]:
#%conda install ipytest
#%pip install ipytest
import ipytest

ipytest.autoconfig()

In [ ]:
%%ipytest -qq

import pytest

from implementations.account_solution import Account
import implementations.portfolio_solution
from implementations.position_solution import Position

import importlib

importlib.reload(implementations.portfolio_solution)


def test_get_all_accounts():
    # GIVEN
    portfolio_name = "TestPortfolio"
    account_a_positions = [
        Position("MSFT US Equity", 1000),
        Position("TSLA US Equity", 2000),
    ]
    account_b_positions = [
        Position("APPL US Equity", 500),
        Position("RIVN US Equity", 1000),
    ]
    account_a = Account(account_a_positions, "Account A")
    account_b = Account(account_b_positions, "Account B")

    accounts = {account_a, account_b}
    expected_accounts = {acc.get_name(): True for acc in accounts}

    # WHEN
    p = implementations.portfolio_solution.Portfolio(portfolio_name, accounts)

    # EXPECT
    all_accs = p.get_all_accounts()

    for acc in all_accs:
        assert acc.get_name() in expected_accounts
        expected_accounts[acc.get_name()] = False

    assert True not in expected_accounts.values()


@pytest.mark.parametrize(
    "input_account, input_security, expected_map",
    (
        ([], [], {"Account A": True, "Account B": True, "Account C": True}),
        (
            ["Account A", "Account B", "Account DNE"],
            [],
            {"Account A": True, "Account B": True},
        ),
        (
            [],
            ["IBM US Equity", "FOOD US Equity"],
            {"Account A": True, "Account B": True, "Account C": True},
        ),
        (["Account B", "Account C"], ["IBM US Equity"], {"Account B": True}),
    ),
)
def test_get_subset_accounts(input_account, input_security, expected_map):
    # GIVEN
    portfolio_name = "TestPortfolio"
    account_a_positions = [
        Position("MSFT US Equity", 1000),
        Position("TSLA US Equity", 2000),
        Position("IBM US Equity", 3000),
    ]
    account_b_positions = [
        Position("APPL US Equity", 500),
        Position("RIVN US Equity", 1000),
        Position("IBM US Equity", 1234),
    ]
    account_c_positions = [
        Position("SWS US Equity", 241),
        Position("CORE US Equity", 4213),
        Position("FOOD US Equity", 1234),
    ]
    account_a = Account(account_a_positions, "Account A")
    account_b = Account(account_b_positions, "Account B")
    account_c = Account(account_c_positions, "Account C")

    accounts = {account_a, account_b, account_c}

    # WHEN
    p = implementations.portfolio_solution.Portfolio(portfolio_name, accounts)

    # EXPECT
    filtered_accounts = p.get_accounts(input_account, input_security)

    for acc in filtered_accounts:
        assert acc.get_name() in expected_map
        expected_map[acc.get_name()] = False

    assert True not in expected_map.values()


def test_add_accounts_no_overwrite():
    # GIVEN
    portfolio_name = "TestPortfolio"
    account_a_positions = [
        Position("MSFT US Equity", 1000),
        Position("TSLA US Equity", 2000),
    ]
    account_b_positions = [
        Position("APPL US Equity", 500),
        Position("RIVN US Equity", 1000),
    ]
    account_a = Account(account_a_positions, "Account A")
    account_b = Account(account_b_positions, "Account B")

    accounts = {account_a, account_b}
    expected_accounts = {acc.get_name(): True for acc in accounts}

    # WHEN
    p = implementations.portfolio_solution.Portfolio(portfolio_name, [])
    p.add_accounts(accounts)

    # EXPECT
    all_accs = p.get_all_accounts()

    for acc in all_accs:
        assert acc.get_name() in expected_accounts
        expected_accounts[acc.get_name()] = False

    assert True not in expected_accounts.values()


def test_add_account_overwrite():
    # GIVEN
    portfolio_name = "TestPortfolio"
    account_a_positions = [
        Position("MSFT US Equity", 1000),
        Position("TSLA US Equity", 2000),
    ]
    account_b_positions = [
        Position("APPL US Equity", 500),
        Position("RIVN US Equity", 1000),
    ]
    account_a = Account(account_a_positions, "Account A")
    account_b = Account(account_b_positions, "Account B")
    accounts = {account_a, account_b}

    # WHEN
    p = implementations.portfolio_solution.Portfolio(portfolio_name, accounts)
    account_b_positions_new = [
        Position("PELO US Equity", 500),
        Position("IBM US Equity", 1000),
    ]
    account_b_new = Account(account_b_positions_new, "Account B")
    expected_accounts = {
        "Account A": account_a_positions,
        "Account B": account_b_positions_new,
    }
    p.add_accounts([account_b_new])

    # EXPECT
    all_accs = p.get_all_accounts()

    for acc in all_accs:
        assert acc.get_name() in expected_accounts
        pos_expected = {
            x.get_security().get_name(): x.get_position()
            for x in expected_accounts[acc.get_name()]
        }
        return_pos = acc.get_all_positions()

        for pos in return_pos:
            assert pos.get_security().get_name() in pos_expected
            assert (
                pos_expected[pos.get_security().get_name()]
                == pos.get_position()
            )

            # Remove the validated Position from our expected map.
            del pos_expected[pos.get_security().get_name()]

        assert len(pos_expected) == 0


def test_remove_accounts():
    # GIVEN
    portfolio_name = "TestPortfolio"
    account_a_positions = [
        Position("MSFT US Equity", 1000),
        Position("TSLA US Equity", 2000),
    ]
    account_b_positions = [
        Position("APPL US Equity", 500),
        Position("RIVN US Equity", 1000),
    ]
    account_a = Account(account_a_positions, "Account A")
    account_b = Account(account_b_positions, "Account B")

    accounts = {account_a, account_b}
    expected_accounts = {"Account A": True}

    # WHEN
    p = implementations.portfolio_solution.Portfolio(portfolio_name, accounts)
    p.remove_accounts(["Account B", "Account DNE"])

    # EXPECT
    all_accs = p.get_all_accounts()

    for acc in all_accs:
        assert acc.get_name() in expected_accounts
        expected_accounts[acc.get_name()] = False

    assert True not in expected_accounts.values()